# Prezentacja 3

In [1]:
from pandas import read_csv, to_datetime, read_excel, Series, date_range
import numpy as np
from scipy.stats import t, norm

###  Notowania giełdowe PEPSICO (PEP) - od 13.04.2015 do 31.03.2025, co tydzień

https://www.nasdaq.com/market-activity/stocks/pep/historical?page=1&rows_per_page=10&timeline=y10

In [27]:
dane_notowania = read_csv('dane\\notowania.csv')
dane_notowania['Date'] = to_datetime(dane_notowania['Date'], format='%m/%d/%Y')
dane_notowania = dane_notowania.sort_values('Date')
dane_notowania.set_index('Date', inplace=True)
# Ustalenie startowej daty: 13.04.2015
start_date = to_datetime('2015-05-10')
dane_notowania = dane_notowania[dane_notowania.index >= start_date]
dane_notowania = dane_notowania.resample('7D').first()
daty_notowania = dane_notowania.index.tolist()
dane_notowania = dane_notowania["Close/Last"].tolist()
dane_notowania = [float(x.replace('$', '')) for x in dane_notowania]
# aby uniknąć problemu związanemu z brakiem danych w weekendy i dni niehandlowe, bierzemy notowanie z każdego poniedziałku (jeśli nie ma z danego poniedziałku, to z wtorku etc.)

## VaR dla notowań

### Metoda parametryczna

In [28]:
# Obliczanie tygodniowych zmian procentowych
zmiany_procentowe = Series(dane_notowania).pct_change().dropna() * 100
daty_zmian = daty_notowania[1:]  # pierwszy tydzień nie ma zmiany

# Unormowane dane lub oryginalne zmiany procentowe:
data = -np.array(zmiany_procentowe)

# Dopasuj rozkład t: zwraca df, loc, scale
df_fit, loc_fit, scale_fit = t.fit(data)

print("VaR rzędu 95% dla notowań:", t.ppf(0.95, df_fit, loc=loc_fit, scale=scale_fit))
print("VaR rzędu 99% dla notowań:", t.ppf(0.99, df_fit, loc=loc_fit, scale=scale_fit))

VaR rzędu 95% dla notowań: 3.514487206967012
VaR rzędu 99% dla notowań: 5.876445890530571


### Metoda historyczna (zwykła)

In [29]:
print("Kwantyl empiryczny dla danych:", np.quantile(data, 0.95))
print("Kwantyl empiryczny dla danych:", np.quantile(data, 0.99))

Kwantyl empiryczny dla danych: 3.5473920358945366
Kwantyl empiryczny dla danych: 5.6245881905998685


In [65]:
def weighted_historical_VaR(data, alpha, lambd=0.999):
    n = len(data)
    w_1 = (1 - lambd) / (1 - np.power(lambd, n))
    weights = np.power(lambd, n - np.arange(1, n + 1)) * w_1
    return np.quantile(data, alpha, weights=weights, method="inverted_cdf")

print(weighted_historical_VaR(data, 0.95))
print(weighted_historical_VaR(data, 0.99))

1.5973826020015403
3.0062630480166996


### VaR obliczony z użyciem filtrowania szeregiem GARCH

In [52]:
from arch import arch_model


# === 1. Dopasowanie modelu GARCH do zwrotów ===
model = arch_model(data, vol='Garch', p=1, q=1, dist='t')
res = model.fit(disp='off')

mu_hat = res.params['mu']
sigma_t = res.conditional_volatility
L = data

standardized_residuals = (L - mu_hat) / sigma_t

alpha = 0.95
q_alpha = np.quantile(standardized_residuals, alpha)

forecast = res.forecast(horizon=1)
sigma_n1 = np.sqrt(forecast.variance.values[-1, : ])[0]

# === 5. Oblicz VaR ===
VaR_alpha = sigma_n1 * q_alpha + mu_hat

print(f"📉 Value at Risk (VaR) na poziomie {alpha*100}%: {VaR_alpha:.4f}")

📉 Value at Risk (VaR) na poziomie 95.0%: 5.0414


ValueError: parameters must have 1 elements

In [67]:
dane_cukier = [
    2.55,2.52,2.45,2.45,2.41,2.38,2.33,2.69,2.89,2.86,2.77,2.86, # 2010
    3.01,3.43,4.71,4.72,4.33,3.93,3.93,3.61,3.68,3.61,3.72,3.61, # 2011
    3.78,3.79,3.79,3.84,3.80,3.84,3.80,3.77,3.78,3.73,3.78,3.76, # 2012
    3.67,3.53,3.67,3.55,3.55,3.47,3.33,3.22,3.14,3.15,3.01,2.85, # 2013
    2.83,2.73,2.58,2.35,2.18,2.24,2.04,2.16,1.93,1.98,1.89,1.81, # 2014
    1.85,1.85,1.98,2.00,1.98,1.94,2.20,2.21,2.30,2.28,2.33,2.34, # 2015
    2.32,2.55,2.60,2.61,2.67,2.65,2.91,2.82,3.07,2.96,3.11,2.98, # 2016
    3.16,2.95,2.93,3.14,2.97,3.12,2.97,3.06,2.84,2.71,2.87,2.46, # 2017
    2.02,2.48,1.96,2.41,1.96,1.91,2.14,2.00,2.03,2.21,2.06,1.87, # 2018
    2.36,2.36,2.57,2.34,2.65,2.37,2.62,2.47,2.68,2.42,2.73,2.42, # 2019
    2.70,2.50,2.54,2.63,2.72,2.66,2.47,2.65,2.38,2.38,2.59,2.57, # 2020
    2.75,2.42,2.53,2.81,2.61,2.51,3.01,2.44,2.53,3.11,3.03,3.33, # 2021
    3.06,3.06,3.33,3.34,3.93,3.33,4.25,5.49,6.76,5.65,6.64,6.00, # 2022
    6.68,6.02,6.63,6.02,6.59,5.90,6.54,5.81,6.14,5.60,6.03,5.70, # 2023
    6.12,4.76,4.90,5.09,4.55,5.31,4.40,4.86,3.65,4.53,3.51,3.30] # 2024

# Obliczanie tygodniowych zmian procentowych
zmiany_procentowe = Series(dane_cukier).pct_change().dropna() * 100

data = -np.array(zmiany_procentowe)

df_fit, loc_fit, scale_fit = t.fit(data)

print("VaR rzędu 95% dla notowań:", t.ppf(0.95, df=df_fit, loc=loc_fit, scale=scale_fit))
print("VaR rzędu 99% dla notowań:", t.ppf(0.99, df=df_fit, loc=loc_fit, scale=scale_fit))

print("Kwantyl empiryczny dla danych:", np.quantile(data, 0.95))
print("Kwantyl empiryczny dla danych:", np.quantile(data, 0.99))

print(weighted_historical_VaR(data, 0.95))
print(weighted_historical_VaR(data, 0.99))

# === 1. Dopasowanie modelu GARCH do zwrotów ===
model = arch_model(data, vol='Garch', p=1, q=1, dist='t')
res = model.fit(disp='off')

mu_hat = res.params['mu']
sigma_t = res.conditional_volatility
L = data

standardized_residuals = (L - mu_hat) / sigma_t

alpha_1 = 0.95
alpha_2 = 0.99
q_alpha_1 = np.quantile(standardized_residuals, alpha_1)
q_alpha_2 = np.quantile(standardized_residuals, alpha_2)

forecast = res.forecast(horizon=1)
sigma_n1 = np.sqrt(forecast.variance.values[-1, : ])[0]

# === 5. Oblicz VaR ===
VaR_alpha_1 = sigma_n1 * q_alpha_1 + mu_hat
VaR_alpha_2 = sigma_n1 * q_alpha_2 + mu_hat

print(f"📉 Value at Risk (VaR) na poziomie {alpha_1*100}%: {VaR_alpha_1:.4f}")
print(f"📉 Value at Risk (VaR) na poziomie {alpha_2*100}%: {VaR_alpha_2:.4f}")

VaR rzędu 95% dla notowań: 16.041620735736437
VaR rzędu 99% dla notowań: 27.47799656916375
Kwantyl empiryczny dla danych: 15.382469849586696
Kwantyl empiryczny dla danych: 22.2869757174393
16.420118343195256
22.51655629139073
📉 Value at Risk (VaR) na poziomie 95.0%: 20.5352
📉 Value at Risk (VaR) na poziomie 99.0%: 29.5133


## Testy dla wartości zagrożonej

### Test kupca

### Test niezależności (Christoffersena) 